In [1]:
import copy
import numpy as np
from scipy.linalg import norm
from scipy import linalg 
import math 

#### Решение методом Гаусса

Входные параметры: 

In [49]:
n = 10
A = [[0] * (n + 1) for i in range(n)]
for i in range(n):
    for j in range(n + 1):
        for k in range(n):
            A[k][n] = 1/(k+1)  
        if i == j:
            A[i][j] = 1
        else:
            A[i][j] = 1/(i+j+2)


In [45]:
def gaussFunc(a):
    eps = 1e-16

    c = numpy.array(a)
    a = numpy.array(a)

    len1 = len(a[:, 0])
    len2 = len(a[0, :])
    vectB = copy.deepcopy(a[:, len1])

    for g in range(len1):

        max = abs(a[g][g])
        my = g
        t1 = g
        while t1 < len1:
            # for t1 in range(len(a[:,0])):
            if abs(a[t1][g]) > max:
                max = abs(a[t1][g])
                my = t1
            t1 += 1

        if abs(max) < eps:
            raise DetermExeption("Check determinant")

        if my != g:
            # a[g][:], a[my][:] = a[my][:], a[g][:]
            # numpy.swapaxes(a, 1, 0)
            b = copy.deepcopy(a[g])
            a[g] = copy.deepcopy(a[my])
            a[my] = copy.deepcopy(b)

        amain = float(a[g][g])

        z = g
        while z < len2:
            a[g][z] = a[g][z] / amain
            z += 1

        j = g + 1

        while j < len1:
            b = a[j][g]
            z = g

            while z < len2:
                a[j][z] = a[j][z] - a[g][z] * b
                z += 1
            j += 1

    a = backTrace(a, len1, len2)


    return a


class DetermExeption(Exception):
    def __init__(self, value):
        self.value = value

    def __str__(self):
        return repr(self.value)


def backTrace(a, len1, len2):
    a = numpy.array(a)
    i = len1 - 1
    while i > 0:
        j = i - 1

        while j >= 0:
            a[j][len1] = a[j][len1] - a[j][i] * a[i][len1]
            j -= 1
        i -= 1
    return a[:, len2 - 1]


def vectorN(c, a, len1, vectB):  # c-начальная матрица a-ответ len-ранг, vectB-вектор B
    c = numpy.array(c)
    a = numpy.array(a)
    vectB = numpy.array(vectB)

    b = numpy.zeros((len1))

    i = 0


    while i<len1:
        j = 0
        while j<len1:

            b[i]+=c[i][j]*a[j]

            j+=1

        i=i+1



    c = copy.deepcopy(b)
    print("!")

    for i in range(len1):
        c[i] = abs(c[i] - vectB[i])

    return c

In [47]:

b = numpy.array(A)
q = gaussFunc(b)            
#for row in a:
#    print(' '.join([str(elem) for elem in row]))
print('\n Ответ:', q)


 Ответ: [ 0.92123854  0.17754425  0.06592651  0.02922762  0.0133203   0.00534375
  0.00097653 -0.00155032 -0.00306143 -0.05911641]


##### Невязка:

In [69]:
i=0
x = numpy.zeros((n))
while i<(n):
        j = 0
        while j<(n):
            x[i]+=a[i][j]*q[j]  
            j+=1
        i=i+1
       
print('Ax =',x)
i=0
while i<(n):
    x[i]=abs(x[i]-1/(i+1))  
    i+=1

f=numpy.zeros(n+1)
i=0
while i<(n+1):
    f[i]=1/(i+1)
    i+=1
print('\nf =', f) 

print('\n')        
print('Ax-f =',x)

print('\n')



Ax = [1.         0.5        0.33333333 0.25       0.2        0.16666667
 0.14285714 0.125      0.11111111 0.04761905]

f = [1.         0.5        0.33333333 0.25       0.2        0.16666667
 0.14285714 0.125      0.11111111 0.1        0.09090909]


Ax-f = [1.11022302e-16 1.11022302e-16 5.55111512e-17 1.11022302e-16
 0.00000000e+00 0.00000000e+00 0.00000000e+00 4.16333634e-17
 1.38777878e-17 5.23809524e-02]




#### Итерационный метод Зейделя $u^{(k+1)}=-(L+D)^{-1}Uu^{(k)}+(L+D)^{-1}f$

In [52]:
eps = 0.0001
n=10
b = [[1/i]  for i in range(1,n+1)]
a = [[0] * (n) for i in range(0,n) ]
for i in range(n):
    for j in range(n):
        if i == j:
            a[i][j] = 1
        else:
            a[i][j] = 1/(i+j+2)
#for row in a:
#    print(' '.join([str(elem) for elem in row]))
#print('\n')


In [188]:

LD = [[0] * (n) for i in range(n) ]
for i in range(n):
    for j in range(n):
        if i == j:
            LD[i][j] = 1
        elif i > j:
            LD[i][j] = round(1/(i+j+2),2)
        else:
            LD[i][j] = 0

U = [[0] * (n) for i in range(n) ]
for i in range(n):
    for j in range(n):
        if i < j:
            U[i][j] = round(1/(i+j+2), 10)
        else:
            U[i][j] = 0

R = -np.dot(linalg.inv(LD), U)
F = np.dot(linalg.inv(LD), b)

u = [.0  for i in range(0,n*n)]
u[0] = np.dot(R,F) + F

eps = 0.00001
i = 0
converge = False
while not converge:     
    u[i + 1] = np.dot(R, u[i]) + F
    converge = numpy.linalg.norm((u[i+1] - u[i]))  <= eps
    i += 1
print('\n Ответ',  u[i] )


 Ответ [[ 0.91769231]
 [ 0.17873621]
 [ 0.06320132]
 [ 0.02615692]
 [ 0.0083975 ]
 [ 0.00690428]
 [ 0.00391325]
 [-0.00248973]
 [-0.0048061 ]
 [-0.00462548]]


#### Невязка:

In [68]:
i=0
y = numpy.zeros((n))
while i<(n):
        j = 0
        while j<(n):
            y[i]+=a[i][j]*w[j]  
            j+=1
        i=i+1
   
        
print('Ax =',b)
i=0
while i<(n):
    y[i]=abs(y[i]-1/(i+1))  
    i+=1

print('\nf =', b) 

print('\n')        
print('Ax-f =',y)

print('\n')


Ax= [[1.0], [0.5], [0.3333333333333333], [0.25], [0.2], [0.16666666666666666], [0.14285714285714285], [0.125], [0.1111111111111111], [0.1]]

f = [[1.0], [0.5], [0.3333333333333333], [0.25], [0.2], [0.16666666666666666], [0.14285714285714285], [0.125], [0.1111111111111111], [0.1]]


Ax-f = [5.31241717e-12 3.05897527e-03 1.33393296e-12 4.15211715e-04
 2.20323437e-03 3.59351453e-03 4.81758429e-03 1.13528367e-03
 2.94369028e-04 1.25603748e-03]




#### Собственные значения матрицы $A$

Наибольшее собственное значение матрицы $A$ по формуле $\dfrac{(u^{(k+1)}u^{(k)})}{(u^{(k)},u^{(k)})}$

In [195]:
n = 10
t = [[0]*(n) for i in range(n+1) ]
t[0] = [1  for i in range(n) ]#Первое приближение

#D, V=numpy.linalg.eig(a)
#print(D)
#print('lambda_max',max(D))
#print('lambda_min',min(D)) 

i=0
j=1
eps = 0.00001
converge = False
while not converge:
    
    t[i+1] = np.dot(a, t[i])
    t[i+2] = np.dot(a, t[i+1])
    li = np.dot(t[i+1], t[i])/ np.dot(t[i], t[i])
    lj = np.dot(t[i+2], t[i+1])/ np.dot(t[i+1], t[i+1])
    converge = numpy.linalg.norm(li-lj)  <= eps
    i += 1
    j += 1
print('\n max собственное значение =', lj)



 max собственное значение = 2.048357259579568


Найдем минимальное собственное значение как $\dfrac{1}{q_{max}}$, где $q_{max}$ - максимальное собственное значение матрицы $A^{-1}$

In [194]:
n = 10
t = [[0]*(n+1) for i in range(n*n) ]
t[0] = [1  for i in range(n) ]#Первое приближение

r = linalg.inv(a)

i=0
j=1
eps = 0.00001
converge = False
while not converge:
    
    t[i+1] = np.dot(r, t[i])
    t[j+1] = np.dot(r, t[i+1])
    qi = np.dot(t[i+1], t[i])/ np.dot(t[i], t[i])
    qj = np.dot(t[j+1], t[i+1])/ np.dot(t[i+1], t[i+1])
    converge = numpy.linalg.norm(li-lj)  <= eps
    i += 1
    j += 1

print('\n min собственное значение матрицы A = ', 1/qj)


 min собственное значение матрицы A =  1.552397965025233


#### Норма матрицы $A$

Число обусловленности в первой и второй нормах $\|A\|>\dfrac{\|Au\|}{\|u\|}$

In [210]:
e  = [1  for i in range(n) ]
print(linalg.norm(np.dot(a, e), ord=1)/linalg.norm(e, ord=1)*linalg.norm(np.dot(r, e), ord=1)/linalg.norm(e, ord=1))
print(linalg.norm(np.dot(a, e), ord=2)/linalg.norm(e, ord=2)*linalg.norm(np.dot(r, e), ord=2)/linalg.norm(e, ord=2))


1.0427924967893238
1.0946167835545506


Для третей нормы $\|A\|_3=\sqrt{(max\lambda)^2}$

In [197]:
lj*qj

1.319479479958139